In [2]:
import cv2 as cv
import numpy as np

cap = cv.VideoCapture('Data\Videos\wymont.MOV')

# Take first frame and convert it to grayscale
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)

# Create a mask image for drawing purposes
hsv = np.zeros_like(frame1)
hsv[...,1] = 255

while(1):
    ret, frame2 = cap.read()
    next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)

    # Calculate optical flow between two frames
    flow = cv.calcOpticalFlowFarneback(prvs,next,None,0.5,3,15,3,5,1.2,0)

    # Convert optical flow to polar coordinates
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])

    # Set image hue according to the optical flow direction
    hsv[...,0] = ang*180/np.pi/2

    # Set image value according to the optical flow magnitude (normalized)
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)

    # Convert HSV to BGR
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)

    cv.imshow('frame2',bgr)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

    # Update previous frame
    prvs = next

cap.release()
cv.destroyAllWindows()


In [ ]:
import cv2 as cv
import numpy as np

# Load images
img1 = cv.imread('img1.jpg')
img2 = cv.imread('img2.jpg')

# Convert images to grayscale
gray1 = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
gray2 = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)

# Detect keypoints and compute descriptors using SIFT
sift = cv.SIFT_create()
kp1, des1 = sift.detectAndCompute(gray1, None)
kp2, des2 = sift.detectAndCompute(gray2, None)

# Match keypoints using FLANN matcher
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

# Filter matches using Lowe's ratio test
good_matches = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good_matches.append(m)

# Estimate fundamental matrix using RANSAC
pts1 = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
pts2 = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
F, mask = cv.findFundamentalMat(pts1, pts2, cv.FM_RANSAC)

# Estimate essential matrix from fundamental matrix and camera intrinsics
K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
E = K.T @ F @ K

# Decompose essential matrix into rotation and translation matrices
retval, R1, t1, R2, t2 = cv.decomposeEssentialMat(E)

# Triangulate points using the first camera matrix and the second camera matrix
P1 = np.array([[1., 0., 0., 0.], [0., 1., 0., 0.], [0., 0., 1., 0.]])
P2_1 = K @ np.hstack((R1,t1))
P2_2 = K @ np.hstack((R1,t2))
P2_3 = K @ np.hstack((R2,t1))
P2_4 = K @ np.hstack((R2,t2))
points4D_1 = cv.triangulatePoints(P1,P2_1,pts1.T[:3],pts2.T[:3])
points4D_2 = cv.triangulatePoints(P1,P2_2,pts1.T[:3],pts2.T[:3])
points4D_3 = cv.triangulatePoints(P1,P2_3,pts1.T[:3],pts2.T[:3])
points4D_4 = cv.triangulatePoints(P1,P2_4,pts1.T[:3],pts2.T[:3])

# Convert homogeneous coordinates to Euclidean coordinates
points3D_1 = points4D_1[:3]/points4D_1[3]
points3D_2 = points4D_2[:3]/points4D_2[3]
points3D_3 = points4D_3[:3]/points4D_3[3]
points3D_4 = points4D_4[:3]/points4D_4[3]

print(points3D_1)
print(points3D_2)
print(points3D_3)
print(points3D_4)


In [ ]:
import cv2 as cv
import numpy as np

# Load image
img = cv.imread('img.jpg')

# Create mask
mask = np.zeros_like(img[:,:,0])
mask[100:200, 100:200] = 255

# Convert image to grayscale
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# Detect corners using goodFeaturesToTrack with mask
corners = cv.goodFeaturesToTrack(gray, 100, 0.01, 10, mask=mask)

# Draw corners on image
for corner in corners:
    x,y = corner.ravel()
    cv.circle(img,(x,y),3,(0,255,0),-1)

# Display image
cv.imshow('image', img)
cv.waitKey(0)
cv.destroyAllWindows()
